In [1]:
import requests
from bs4 import BeautifulSoup
from fake_useragent import UserAgent
import re
from tqdm import tqdm
from time import sleep
from nltk.tokenize import word_tokenize
import pymorphy2
from random import uniform
from string import punctuation
from collections import Counter
from sklearn.metrics import accuracy_score
from nltk.corpus import stopwords

sw = stopwords.words("russian") + ["всё", "самый", "это"]

punctuation += "–«»…"

morph = pymorphy2.MorphAnalyzer()

In [2]:
ua = UserAgent(verify_ssl=False)

Соберём с Кинопоиска отзывы на фильмы одного жанра, например, ужасы. Всего 20 фильмов, на каждый возьмём по 10 хороших и 10 плохих отзывов. Из 200 хороших отзывов на 190 соберём данные, на 10 проверим, так же для плохих отзывов.

In [3]:
rev_links = ["https://www.kinopoisk.ru/film/1178137/reviews/?ord=rating&status=", 
             "https://www.kinopoisk.ru/film/1122138/reviews/?ord=rating&status=",
             "https://www.kinopoisk.ru/film/944708/reviews/?ord=rating&status=",
             "https://www.kinopoisk.ru/film/804697/reviews/?ord=rating&status=",
             "https://www.kinopoisk.ru/film/686898/reviews/?ord=rating&status=",
             "https://www.kinopoisk.ru/film/1044906/reviews/?ord=rating&status=",
             "https://www.kinopoisk.ru/film/279095/reviews/?ord=rating&status=",
             "https://www.kinopoisk.ru/film/915111/reviews/?ord=rating&status=",
             "https://www.kinopoisk.ru/film/966995/reviews/?ord=rating&status=",
             "https://www.kinopoisk.ru/film/468994/reviews/?ord=rating&status=",
             "https://www.kinopoisk.ru/film/261005/reviews/?ord=rating&status=",
             "https://www.kinopoisk.ru/film/160931/reviews/?ord=rating&status=",
             "https://www.kinopoisk.ru/film/721153/reviews/?ord=rating&status=",
             "https://www.kinopoisk.ru/film/930534/reviews/?ord=rating&status=",
             "https://www.kinopoisk.ru/film/495892/reviews/?ord=rating&status=",
             "https://www.kinopoisk.ru/film/419209/reviews/?ord=rating&status=",
             "https://www.kinopoisk.ru/film/695548/reviews/?ord=rating&status=",
             "https://www.kinopoisk.ru/film/991225/reviews/?ord=rating&status=",
             "https://www.kinopoisk.ru/film/1112132/reviews/?ord=rating&status=",
             "https://www.kinopoisk.ru/film/404366/reviews/?ord=rating&status="]


negative_rev_links = [rev+"bad" for rev in rev_links]
positive_rev_links = [rev+"good" for rev in rev_links]

# Солнцестояние
# Мы 
# Прочь
# Оно / It Follows 2014
# Оно 2 
# Тихое место
# Кладбище домашних животных
# Лекарство от здоровья
# Живое
# Заклятие
# Последний дом слева
# У холмов есть глаза
# Багровый пик
# Сплит
# Астрал
# Хижина в лесу
# Чужой: Завет
# Проклятие монахини
# Реинкарнация
# Паранормальное явление

In [4]:
def get_reviews(rev_links):    
    
    reviews = []
    c = 0
    
    for film_reviews in tqdm(rev_links):
        
        sleep(uniform(30, 50))
        
        headers = {'User-Agent': ua.random}
        r = requests.get(film_reviews, headers=headers)        
        soup = BeautifulSoup(r.text)

        reviews += soup.find_all('span', {'class':'_reachbanner_'})

        for i in range(10): # на одной странице 10 отзывов
            reviews[c*10+i] = re.sub("<.*?>|\s{1,}", " ", str(reviews[c*10+i]))
         
        #for i in range(len(reviews)): # на одной странице все отзывы
        #    reviews[i] = re.sub("<.*?>|\s{1,}", " ", str(reviews[i]))
            
        c += 1

    return reviews

In [5]:
negative_reviews = get_reviews(negative_rev_links)

100%|██████████████████████████████████████████████████████████████████████████████████| 20/20 [13:39<00:00, 40.98s/it]


In [6]:
positive_reviews = get_reviews(positive_rev_links)

100%|██████████████████████████████████████████████████████████████████████████████████| 20/20 [13:10<00:00, 39.52s/it]


In [7]:
len(positive_reviews), len(negative_reviews)

(200, 200)

In [8]:
def get_lemmas_counter(reviews):
    
    lemmas = []
    
    for review in reviews:

        review = review.lower().translate(str.maketrans('', '', punctuation))
        review = re.sub('\d+', '', review)
        review = re.sub('\s{2,}', ' ', review)
        review = word_tokenize(review.lower())

        for word in review:
            lemma = morph.parse(word)[0].normal_form
            if lemma not in sw:
                lemmas += [lemma]

        
    lemmas = Counter(lemmas)
            
            
    return lemmas

In [9]:
negative_lemmas_counter = get_lemmas_counter(negative_reviews[:190])
positive_lemmas_counter = get_lemmas_counter(positive_reviews[:190])

In [10]:
def remove_rare(lemmas):
    
    count = {k:v for k,v in lemmas.items() if v > 25}
    count = Counter(count)
    
    return count.most_common()

In [11]:
total_neg_counter = remove_rare(negative_lemmas_counter)
total_pos_counter = remove_rare(positive_lemmas_counter)

In [12]:
positive_lemmas = [i[0] for i in total_pos_counter]
negative_lemmas = [i[0] for i in total_neg_counter]

# ищем слова, которые встречаются только в положительных/отрицательных отзывах

excl_positive_lemmas = [i for i in positive_lemmas if i not in negative_lemmas]
excl_negative_lemmas = [i for i in negative_lemmas if i not in positive_lemmas]

#уравняем количество лемм, по которым смотрим, чтобы не было перевеса в какую-либо сторону

if len(excl_positive_lemmas) > len(excl_negative_lemmas):
    excl_positive_lemmas = excl_positive_lemmas[:len(excl_negative_lemmas)]
else:
    excl_negative_lemmas = excl_negative_lemmas[:len(excl_positive_lemmas)]
    
assert len(excl_negative_lemmas) == len(excl_positive_lemmas)

In [37]:
def detect_tone(review, excl_positive_lemmas=excl_positive_lemmas, excl_negative_lemmas=excl_negative_lemmas):
    
    pos, neg = 0, 0
    
    review = review.lower().translate(str.maketrans('', '', punctuation))
    review = re.sub('\d+', '', review)
    review = re.sub('\s{2,}', ' ', review)
    review = word_tokenize(review.lower())

    for word in review:
        lemma = morph.parse(word)[0].normal_form
        if lemma in excl_positive_lemmas:
            pos += 1
        elif lemma in excl_negative_lemmas:
            neg += 1
    
    if pos > neg:
        tone = 'pos'
    elif neg > pos:
        tone = 'neg'
    else:
        tone = 'NA'
            
    return tone

In [14]:
control_texts = negative_reviews[190:] + positive_reviews[190:] # данные, на которых будем проверять
assert len(control_texts) == 20

control_tones = ["neg"]*10 + ["pos"]*10

tones = []

for text in control_texts:
    tones.append(detect_tone(text))
    
for i in range(20):
    print(control_tones[i], ' ', tones[i])
    
print(accuracy_score(tones, control_tones))

neg   neg
neg   neg
neg   neg
neg   pos
neg   pos
neg   pos
neg   neg
neg   pos
neg   NA
neg   pos
pos   pos
pos   neg
pos   pos
pos   pos
pos   pos
pos   pos
pos   pos
pos   pos
pos   NA
pos   neg
0.55


Точность чуть больше половины, что не очень хорошо, учитывая, что рандомным распределением мы могли бы получить эту самую половину. Возможно, что дело во входных данных: всё-таки фильмы, даже если они все одного жанра, достаточно разные, поэтому я попыталась сузить зону изучения до одного фильма. 

Тогда я взяла отзывы на «Отряд самоубийц», поскольку у него много как хороших, так и плохих (>200). Дальше будет то же самое, изменятся только отзывы и чуть-чуть их сбор (то есть кода нового не будет), посмотрим, будет ли так лучше.

In [31]:
def get_reviews(rev_links):    
    
    reviews = []
    #c = 0
    
    for film_reviews in tqdm(rev_links):
        
        #sleep(uniform(30, 50))
        
        headers = {'User-Agent': ua.random}
        r = requests.get(film_reviews, headers=headers)        
        soup = BeautifulSoup(r.text)

        reviews += soup.find_all('span', {'class':'_reachbanner_'})

        #for i in range(10): # на одной странице 10 отзывов
        #    reviews[c*10+i] = re.sub("<.*?>|\s{1,}", " ", str(reviews[c*10+i]))
         
        for i in range(len(reviews)): # на одной странице все отзывы
            reviews[i] = re.sub("<.*?>|\s{1,}", " ", str(reviews[i]))
             
        #c += 1

    return reviews

In [32]:
SSq_positive_rev_links = ["https://www.kinopoisk.ru/film/468522/reviews/ord/rating/status/good/perpage/200/"]
SSq_negative_rev_links = ["https://www.kinopoisk.ru/film/468522/reviews/ord/rating/status/bad/perpage/200/"]

# Отряд самоубийц

In [33]:
SSq_negative_reviews = get_reviews(SSq_negative_rev_links)
SSq_positive_reviews = get_reviews(SSq_positive_rev_links)

100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:03<00:00,  3.62s/it]


In [34]:
len(SSq_negative_reviews), len(SSq_positive_reviews)

(200, 200)

In [35]:
SSq_negative_lemmas_counter = get_lemmas_counter(SSq_negative_reviews[:190])
SSq_positive_lemmas_counter = get_lemmas_counter(SSq_positive_reviews[:190])

SSq_total_neg_counter = remove_rare(SSq_negative_lemmas_counter)
SSq_total_pos_counter = remove_rare(SSq_positive_lemmas_counter)

SSq_positive_lemmas = [i[0] for i in SSq_total_pos_counter]
SSq_negative_lemmas = [i[0] for i in SSq_total_neg_counter]

SSq_excl_positive_lemmas = [i for i in SSq_positive_lemmas if i not in SSq_negative_lemmas]
SSq_excl_negative_lemmas = [i for i in SSq_negative_lemmas if i not in SSq_positive_lemmas]

if len(SSq_excl_positive_lemmas) > len(SSq_excl_negative_lemmas):
    SSq_excl_positive_lemmas = SSq_excl_positive_lemmas[:len(SSq_excl_negative_lemmas)]
else:
    SSq_excl_negative_lemmas = SSq_excl_negative_lemmas[:len(SSq_excl_positive_lemmas)]
    
assert len(SSq_excl_negative_lemmas) == len(SSq_excl_positive_lemmas)

In [38]:
SSq_control_texts = SSq_negative_reviews[190:] + SSq_positive_reviews[190:]
assert len(SSq_control_texts) == 20

control_tones = ["neg"]*10 + ["pos"]*10

tones = []

for text in SSq_control_texts:
    tones.append(detect_tone(text, SSq_excl_positive_lemmas, SSq_excl_negative_lemmas))
    
for i in range(20):
    print(control_tones[i], ' ', tones[i])
    
print(accuracy_score(tones, control_tones))

neg   neg
neg   neg
neg   pos
neg   neg
neg   neg
neg   pos
neg   neg
neg   pos
neg   neg
neg   NA
pos   pos
pos   neg
pos   pos
pos   pos
pos   pos
pos   neg
pos   neg
pos   pos
pos   pos
pos   pos
0.65


Как видно, точность повысилась, хотя и недостаточно для того, чтобы считать результат достаточно хорошим. Тем не менее, как мне кажется, это всё-таки показывает, что чем ýже тема, на которую написаны отзывы, тем проще научиться определять их тон.

### Как можно это улучшить

В первую очередь, нужно больше данных. Проблема возникает в том, что даже для того, чтобы было достаточно информации по одному фильму, мне пришлось взять одну из самых комментируемых картин на Кинопоиске, то есть для менее популярных фильмов даже такого качества опеределения с таким подходом вряд ли удастся добиться. Тем не менее, в имеющихся условиях, можно почистить данные, например, отобрав только прилагательные и наречия, поскольку они чаще всего и выражают субъективную точку зрения.

In [41]:
excl_negative_lemmas[:20], excl_positive_lemmas[:20]

(['абсолютно',
  'час',
  'диалог',
  'логика',
  'старый',
  'например',
  'видимо',
  'отзыв',
  'попытка',
  'мать',
  'положительный',
  'плюс',
  'глупый',
  'както',
  'рецензия',
  'линия',
  'банальный',
  'поведение',
  'никак',
  'картинка'],
 ['настоящий',
  'джеймс',
  'ван',
  'отличный',
  'весьма',
  'снять',
  'сила',
  'заклятие',
  'шьямалан',
  'создать',
  'несмотря',
  'молодой',
  'душа',
  'странный',
  'завет',
  'отметить',
  'удаться',
  'реинкарнация',
  'день',
  'современный'])

Среди топ-20 лемм обоих окрасов, как видно, настроение передаётся именно прилагательными и наречиями («глупый», «банальный», «старый» (?) vs «отличный», «современный», «молодой» (?) или даже «никак» vs «весьма»).

Ещё можно было бы попробовать учитывать вес слова в словаре «окрашенных» слов при подсчёте тех или иных лемм. То есть, например, если в отзыве самая частая положительная лемма 5 раз, и 5 разных лемм из отрицательного словаря, которые не входят даже в топ-20, то это учитывается в равной степени, хотя можно (и, возможно, нужно) было бы это считать как перевес в сторону положительного.

А вообще мне кажется, что в фильмами в принципе сложнее, чем с, например, товарами — тут нет объективных критериев «плохо» и «хорошо». А ещё у Кинопоиска не различаются «сильно» отрицательные и «немного» отрицательные отзывы, как могло бы быть, если бы у них была унифицированная система рейтинга, скажем, от 1 до 5 или от 1 до 10, где можн было бы брать крайности. 